In [11]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [12]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [31]:
from transformers import AutoTokenizer, TFAutoModel, TFBertModel
import torch
import requests
from bs4 import BeautifulSoup
import re

In [37]:
from transformers import AutoConfig, AutoModelForSequenceClassification

In [32]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')



In [38]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [54]:
tokens = tokenizer.encode('This is amazing, I loved it GREAT!', return_tensors='pt')


In [55]:
result = model(tokens)

In [56]:
result.logits

tensor([[-2.5043, -2.8910, -1.2988,  1.4524,  4.2066]],
       grad_fn=<AddmmBackward0>)

In [58]:
int(torch.argmax(result.logits))+1   #rates between 1 to 5, 5 being positive

5

In [82]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [85]:
reviews

['Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 "It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit.",
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black cof

In [86]:
import numpy as np
import pandas as pd

In [87]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [88]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [89]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [90]:
sentiment_score(df['review'].iloc[1])

5

In [91]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [92]:
df

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,Great staff and food. Must try is the pan fri...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,It was ok. The coffee wasn't the best but it w...,3
4,I came to Social brew cafe for brunch while ex...,5
5,We came for brunch twice in our week-long visi...,4
6,I went here a little while ago- a beautiful mo...,2
7,Ron & Jo are on the go down under and Wow! We...,5
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4


In [93]:
df['review'].iloc[3]

"It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit."

In [96]:
df2 = df["sentiment"].mean()
print("Average Rating:",df2)

Average Rating: 4.363636363636363
